Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
import tensorflowjs as tfjs

nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Jacopo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Pre-Processor

In [2]:
def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    stopwords_list.append('rt')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 
    
def remove_mentions(input_text):
    return re.sub(r'@\w+', '', input_text)

def pre_process(input_text):
    output = input_text.apply(lambda x: x.lower())
    output = output.apply(remove_stopwords).apply(remove_mentions)
    output = output.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    return output

Load and Pre-Process data

In [3]:
#download from kaggle
#!curl https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment/downloads/Sentiment.csv/2 > Sentiment.csv
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
# Easy: binary classifier
data = data[data.sentiment != "Neutral"]

data['text'] = pre_process(data['text'])

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
data.head()

/Users/Jacopo/anaconda/envs/TFJS/lib/python2.7/site-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


4472
16986


,text,sentiment
1,catch full gopdebate last night scotts best l...,Positive
3,carly fiorina trending hours debate men jus...,Positive
4,gopdebate w delivered highest ratings histor...,Positive
5,on first day rescind every illegal executive...,Positive
6,liked happy heard going moderator not anymore...,Negative


Tokenizer

In [4]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
input_length = X.shape[1]

Prepare Embedding

In [5]:
embed_dim = 128
embedding = Embedding(max_fatures, embed_dim,input_length = input_length)

Create Model

In [6]:
lstm_out = 196
model = Sequential()
model.add(embedding)
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 21, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Train Model

In [7]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

((7188, 21), (7188, 2))
((3541, 21), (3541, 2))
Epoch 1/7
 - 9s - loss: 0.4458 - acc: 0.8109
Epoch 2/7
 - 8s - loss: 0.3281 - acc: 0.8624
Epoch 3/7
 - 8s - loss: 0.2843 - acc: 0.8804
Epoch 4/7
 - 8s - loss: 0.2547 - acc: 0.8948
Epoch 5/7
 - 8s - loss: 0.2313 - acc: 0.9053
Epoch 6/7
 - 8s - loss: 0.2060 - acc: 0.9139
Epoch 7/7
 - 8s - loss: 0.1885 - acc: 0.9221


Evaluate Model

In [8]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.46
acc: 0.83


Test Model

In [11]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#twt = ['We all are great people']

#pre-processing
twt = pd.DataFrame({'text':twt})
twt['text'] = pre_process(twt['text'])
twt = twt['text'].tolist()
print(twt)

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=input_length, dtype='int32', value=0)
print(twt)

sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['great people even us mean']
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  67  12
  106  77 144]]
[ 0.88495165  0.11504834]
negative


Export Model and Metadata